# Matching and IPW tutorial

In [1]:
import numpy as np
from numpy.random import randn
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression

We define a simulator for vaccine hesitancy data, where all variables are binary. X represents vaccine hesitancy, T if an individual gets vaccinated and Y if they recover.

In [3]:
# def vaccine_hesitancy_SCM (remove_confounding=False, n_samples=1000):
#     # X = vaccine hesitancy:
#     # epsilon_x
#     epsilon_x = np.random.choice(2, n_samples, p = [0.7, 0.3], replace = True)
#     x = epsilon_x
    
#     # T = get vaccinated:
    
#     if remove_confounding:
#         t = np.random.choice(2, n_samples, p = [0.5, 0.5], replace = True)
#     else:
#         # epsilon_t_0 = Ber(0.9):
#         epsilon_t_0 = np.random.choice(2, n_samples, p = [0.1, 0.9], replace = True)
#         # epsilon_t_1 = Ber(0.1):
#         epsilon_t_1 = np.random.choice(2, n_samples, p = [0.9, 0.1], replace = True)
#         # This is just a way to say that P(T|X=1) = P(epsilon_t_1) and P(T|X=0) = P(epsilon_t_0) 
#         t = epsilon_t_1*x + epsilon_t_0*(1-x)
    
#     # Y = recover
#     # epsilons for all combinations of values for X and T
#     # P(Y=1|X=0, T=0)= Ber(0.6)
#     epsilon_y_00 = np.random.choice(2, n_samples, p = [0.4, 0.6], replace = True)
#     # P(Y=1|X=0, T=1)= Ber(0.9)
#     epsilon_y_01 = np.random.choice(2, n_samples, p = [0.1, 0.9], replace = True)
#     # P(Y=1|X=1, T=0)= Ber(0.5)
#     epsilon_y_10 = np.random.choice(2, n_samples, p = [0.5, 0.5], replace = True)
#     # P(Y=1|X=1, T=1)= Ber(0.75)
#     epsilon_y_11 = np.random.choice(2, n_samples, p = [0.25, 0.75], replace = True)
    
#     # This is just a way to say that P(Y|X=0, T=0) = P(epsilon_y_00), P(Y|X=0, T=1) = P(epsilon_y_01) etc
#     y = x*t*epsilon_y_11 + (1-x)*t*epsilon_y_01 + x*(1-t)*epsilon_y_10 + (1-x)*(1-t)*epsilon_y_00
#     df = pd.DataFrame({ "x": x, "t": t, "y": y})
#     return df

We start by simulating some observational data for vaccine hesitancy, in which people who are not hesitant are also engaging in other behaviours that might help them recover. We compute the Average Treatment Effect (ATE) with these data in a naive way:

In [6]:
# Simulate some observational data:
#df = vaccine_hesitancy_SCM(n_samples=5000)
df=pd.read_csv('lalonde_data.csv')
df.head()


,treat,age,education,black,hispanic,married,nodegree,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.7899


In [9]:
# Vaccinated people:
treatment_group = df[df["treat"]==1]
# Non-vaccinated people:
control_group = df[df["treat"]==0]

# if len(treatment_group[treatment_group["x"]==1]) == 0 or len(treatment_group[treatment_group["x"]==0]) == 0:
#     if len(control_group[control_group["x"]==1]) == 0 or len(control_group[control_group["x"]==0]) == 0:
#         print("Positivity is violated, simulate more samples.")

ATE_biased = np.mean(treatment_group["re78"]) - np.mean(control_group["re78"])

print("ATE in observational data:", ATE_biased)

ATE in observational data: 886.3037307037439


We now simulate some data without any confounding, e.g. what we would get by running a randomized control trial and randomly assigning people to get vaccinated or not. We then compute the Average Treatment Effect (ATE) with these data, which is an unbiased estimate of the real ATE:

In [11]:
# df_rct = vaccine_hesitancy_SCM(n_samples=10000, remove_confounding=True)
# # Vaccinated people:
# treatment_group_rct = df_rct[df_rct["t"]==1]
# # Non-vaccinated people:
# control_group_rct = df_rct[df_rct["t"]==0]

# ATE_rct = np.mean(treatment_group_rct["y"]) - np.mean(control_group_rct["y"])

# print("ATE in an uncofounded setting (e.g. randomized controlled trial (RCT)):", ATE_rct, "(should be close to 0.3)")

In [12]:
df.head()

,treat,age,education,black,hispanic,married,nodegree,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.7899


We can see that the estimates for the ATEs are fairly different in the two settings (and we know the ATE in the RCT is supposed to be an approximation of the true ATE, which is close to 0.3), and that the ATE without correction overestimates the effect of the vaccine.

## Exact matching:

We can now conceptually pair each individual in the treatment group with $X=x$ with an individual in the control group with $X=x$, and remove the ones which don't match.

In [14]:
treatment_group_x_0 = treatment_group[treatment_group["nodegree"]==0]
treatment_group_x_1 = treatment_group[treatment_group["nodegree"]==1]

control_group_x_0 = control_group[control_group["nodegree"]==0]
control_group_x_1 = control_group[control_group["nodegree"]==1]

print("Number of people with nodegree=0 in treatment: ", len(treatment_group_x_0)," and in control: ", len(control_group_x_0))
print("Number of people with nodegree=1 in treatment: ", len(treatment_group_x_1)," and in control: ", len(control_group_x_1))

Number of people with nodegree=0 in treatment:  80  and in control:  79
Number of people with nodegree=1 in treatment:  217  and in control:  346


The numbers of people with nodegree=0 and nodegree=1 is quite unbalanced in the control and treatment.
We can also check covariate balancing on the original data P(X|T=0) should the same as P(X|T=1), or in finite data relatively close (we can use standardized mean difference to check how close they are in distribution). Since X is a binary variable, we can just check what happens for X=0

In [15]:
# approximating P(X=0|T=1)
freq_x0_treatment = len(treatment_group_x_0)/len(treatment_group)
# approximating P(X=0|T=0)
freq_x0_control = len(control_group_x_0)/len(control_group)
# In order to be balanced these two should be very close :
print("Proportion of nodegree=0 on total in treatment: ", freq_x0_treatment, " and in control: ", freq_x0_control)

Proportion of nodegree=0 on total in treatment:  0.26936026936026936  and in control:  0.18588235294117647


We can now perform the matching. Since all units with X=x are conceptually the same, we don't need to actually match the pairs, but we can just keep the same number of people for treatment and control groups for each value of X=x.

In [17]:
min_number_x0 = min(len(treatment_group_x_0), len(control_group_x_0))
balanced_treatment_x_0 = treatment_group_x_0[0:min_number_x0]
balanced_control_x_0 = control_group_x_0[0:min_number_x0]
print("After balancing: number of people with nodegree=0 in treatment: ", len(balanced_treatment_x_0)," and in control: ", len(balanced_control_x_0))

min_number_x1 = min(len(treatment_group_x_1), len(control_group_x_1))
balanced_treatment_x_1 = treatment_group_x_1[0:min_number_x1]
balanced_control_x_1 = control_group_x_1[0:min_number_x1]
print("After balancing: number of people with nodegree=1 in treatment: ", len(balanced_treatment_x_1)," and in control: ", len(balanced_control_x_1))

After balancing: number of people with nodegree=0 in treatment:  79  and in control:  79
After balancing: number of people with nodegree=1 in treatment:  217  and in control:  217


We can now check again the covariate balancing and see that they are balanced and then just use this smaller dataset to estimate the ATE (which turns out to be closer to the true value than the original confounded case):

In [19]:
balanced_treatment = pd.concat([balanced_treatment_x_0, balanced_treatment_x_1])
balanced_control = pd.concat([balanced_control_x_1, balanced_control_x_0])

# approximating P(X=0|T=1)
freq_x0_treatment_matching = len(balanced_treatment_x_0)/len(balanced_treatment)
# approximating P(X=0|T=0)
freq_x0_control_matching = len(balanced_control_x_0)/len(balanced_control)
print("Proportion of X=0 on total in treatment: ", freq_x0_treatment_matching, " and in control: ", freq_x0_control_matching)

ATE_matching = np.mean(balanced_treatment["re78"]) - np.mean(balanced_control["re78"])
print("ATE after balancing:", ATE_matching)

Proportion of X=0 on total in treatment:  0.2668918918918919  and in control:  0.2668918918918919
ATE after balancing: 701.6755812837837


We now used only one variable X for simplicity as the adjustment set. In general this method works also with more than one variable in the adjustment set, and if the variables are not discrete can also use a distance metric to match the units. Unfortunately, this method discared a lot of data, making the estimation less stable.

## Propensity scores:
We now compute $P(T=1|X)$ for each value of $X$:

In [21]:
x0_group = pd.concat([treatment_group_x_0,control_group_x_0])
x1_group = pd.concat([treatment_group_x_1,control_group_x_1])

propensity_score_x0 = len(treatment_group_x_0)/len(x0_group)
propensity_score_x1 = len(treatment_group_x_1)/len(x1_group)

print(propensity_score_x0, ": true value 0.5, ",  propensity_score_x1, ": true value 0.4")

0.5031446540880503 : true value 0.5,  0.38543516873889877 : true value 0.4


We could do matching on the values of propensity scores (this is called propensity score matching), but in this case it doesn't really improve things since the probabilities are quite different for every combination of X. Otherwise it can be seen as a way of clustering values of X that have a similar effect on T.

## Inverse probability weighting

Instead we just use the formula to compute a weighted average for treatment and control:

In [22]:
number_of_all_samples = len(df)

weighted_sum_treatment_0 = sum(treatment_group_x_0["re78"])/  propensity_score_x0
weighted_sum_treatment_1 = sum(treatment_group_x_1["re78"])/ propensity_score_x1

mean_treatment = (weighted_sum_treatment_0 + weighted_sum_treatment_1) /number_of_all_samples

weighted_sum_control_0 = sum(control_group_x_0["re78"])/(1 - propensity_score_x0)
weighted_sum_control_1 = sum(control_group_x_1["re78"])/ (1 - propensity_score_x1)

mean_control = (weighted_sum_control_0 + weighted_sum_control_1)/number_of_all_samples

ATE_IPW = mean_treatment-mean_control
print("ATE after IPW:", ATE_IPW)

ATE after IPW: 784.0364507960767


## Testing the mean and variance of the estimators

In [23]:
def IPW_XTY(df):
    # Vaccinated people:
    treatment_group = df[df["treat"]==1]
    # Non-vaccinated people:
    control_group = df[df["treat"]==0]
    
    number_of_all_samples = len(df)
    treatment_group_x_0 = treatment_group[treatment_group["nodegree"]==0]
    treatment_group_x_1 = treatment_group[treatment_group["nodegree"]==1]

    control_group_x_0 = control_group[control_group["nodegree"]==0]
    control_group_x_1 = control_group[control_group["nodegree"]==1]

    weighted_sum_treatment_0 = sum(treatment_group_x_0["re78"])/  propensity_score_x0
    weighted_sum_treatment_1 = sum(treatment_group_x_1["re78"])/ propensity_score_x1

    mean_treatment = weighted_sum_treatment_0 + weighted_sum_treatment_1
    mean_treatment = mean_treatment/number_of_all_samples

    weighted_sum_control_0 = sum(control_group_x_0["re78"])/(1 - propensity_score_x0)
    weighted_sum_control_1 = sum(control_group_x_1["re78"])/ (1 - propensity_score_x1)

    mean_control = weighted_sum_control_0 + weighted_sum_control_1 
    mean_control = mean_control/number_of_all_samples

    return mean_treatment-mean_control

def exactMatching_XTY(df):
    # Vaccinated people:
    treatment_group = df[df["treat"]==1]
    # Non-vaccinated people:
    control_group = df[df["treat"]==0]
    
    treatment_group_x_0 = treatment_group[treatment_group["re78"]==0]
    treatment_group_x_1 = treatment_group[treatment_group["re78"]==1]

    control_group_x_0 = control_group[control_group["treat"]==0]
    control_group_x_1 = control_group[control_group["treat"]==1]
    
    min_number_x0 = min(len(treatment_group_x_0), len(control_group_x_0))
    balanced_treatment_x_0 = treatment_group_x_0[0:min_number_x0]
    balanced_control_x_0 = control_group_x_0[0:min_number_x0]
    
    min_number_x1 = min(len(treatment_group_x_1), len(control_group_x_1))
    balanced_treatment_x_1 = treatment_group_x_1[0:min_number_x1]
    balanced_control_x_1 = control_group_x_1[0:min_number_x1]
    
    return np.mean(balanced_treatment["treat"]) - np.mean(balanced_control["treat"])

ATE_IPW_list = []
ATE_matching_list = []
ATE_rct_list = []


for i in range(0,100):
    #df = vaccine_hesitancy_SCM(n_samples=5000)
    ATE_IPW_list.append(IPW_XTY(df))
    ATE_matching_list.append(exactMatching_XTY(df))
    #df_rct = vaccine_hesitancy_SCM(n_samples=5000, remove_confounding=True)
    df_rct=df
    # Vaccinated people:
    treatment_group_rct = df_rct[df_rct["treat"]==1]
    # Non-vaccinated people:
    control_group_rct = df_rct[df_rct["treat"]==0]
    ATE_rct_list.append(np.mean(treatment_group_rct["re78"]) - np.mean(control_group_rct["re78"]))


print(np.mean(ATE_IPW_list), np.std(ATE_IPW_list), np.mean(ATE_matching_list), np.std(ATE_matching_list), np.mean(ATE_rct_list), np.std(ATE_rct_list))

784.0364507960767 0.0 1.0 0.0 886.3037307037439 0.0


In [24]:
df.head()

,treat,age,education,black,hispanic,married,nodegree,re75,re78
0,1.0,37.0,11.0,1.0,0.0,1.0,1.0,0.0,9930.0460
1,1.0,22.0,9.0,0.0,1.0,0.0,1.0,0.0,3595.8940
2,1.0,30.0,12.0,1.0,0.0,0.0,0.0,0.0,24909.4500
3,1.0,27.0,11.0,1.0,0.0,0.0,1.0,0.0,7506.1460
4,1.0,33.0,8.0,1.0,0.0,0.0,1.0,0.0,289.7899


## Example with more variables:

In [25]:
# def more_variables_SCM (remove_confounding=False, n_samples=1000):
#     epsilon_x1 = np.random.choice(2, n_samples, p = [0.6, 0.4], replace = True)
#     x1 = epsilon_x1
    
#     epsilon_x2 = np.random.choice(2, n_samples, p = [0.7, 0.3], replace = True)
#     x2 = epsilon_x2
    
#     if remove_confounding:
#         t = np.random.choice(2, n_samples, p = [0.5, 0.5], replace = True)
#     else:
#         epsilon_t_00 = np.random.choice(2, n_samples, p = [0.1, 0.9], replace = True)
#         epsilon_t_10 = np.random.choice(2, n_samples, p = [0.8, 0.2], replace = True)
#         epsilon_t_11 = np.random.choice(2, n_samples, p = [0.95, 0.05], replace = True)
#         epsilon_t_01 = np.random.choice(2, n_samples, p = [0.7, 0.3], replace = True)    
#         t = epsilon_t_00*(1-x1)*(1-x2) + epsilon_t_01*(1-x1)*x2 + epsilon_t_10*x1*(1-x2) + epsilon_t_11*x1*x2
    
    
#     x = x1 * x2
#     epsilon_y_00 = np.random.choice(2, n_samples, p = [0.4, 0.6], replace = True)
#     epsilon_y_01 = np.random.choice(2, n_samples, p = [0.1, 0.9], replace = True)
#     epsilon_y_10 = np.random.choice(2, n_samples, p = [0.5, 0.5], replace = True)
#     epsilon_y_11 = np.random.choice(2, n_samples, p = [0.25, 0.75], replace = True)
#     y = x*t*epsilon_y_11 + (1-x)*t*epsilon_y_01 + x*(1-t)*epsilon_y_10 + (1-x)*(1-t)*epsilon_y_00
    
#     df = pd.DataFrame({ "x1": x1, "x2": x2, "t": t, "y": y})
#     return df

# Simulate some observational data:
#df2 = more_variables_SCM(n_samples=5000)
df2=df

# Vaccinated people:
t_group = df2[df2["treat"]==1]
# Non-vaccinated people:
c_group = df2[df2["treat"]==0]

ATE_biased = np.mean(t_group["re78"]) - np.mean(c_group["re78"])
#df2_rct = more_variables_SCM(n_samples=10000, remove_confounding=True)
# Vaccinated people:
#t_rct = df2_rct[df2_rct["t"]==1]
# Non-vaccinated people:
#c_rct = df2_rct[df2_rct["t"]==0]
#print("ATE in an uncofounded setting (e.g. randomized controlled trial (RCT)):", np.mean(t_rct["y"]) - np.mean(c_rct["y"]), ", unadjusted ATE:", ATE_biased)

treatment_group_x_0 = t_group[t_group["nodegree"]==0]
treatment_group_x_1 = t_group[t_group["nodegree"]==1]

treatment_group_x_00 = treatment_group_x_0[treatment_group_x_0["black"]==0]
treatment_group_x_10 = treatment_group_x_1[treatment_group_x_1["black"]==0]
treatment_group_x_01 = treatment_group_x_0[treatment_group_x_0["black"]==1]
treatment_group_x_11 = treatment_group_x_1[treatment_group_x_1["black"]==1]

control_group_x_0 = c_group[c_group["nodegree"]==0]
control_group_x_1 = c_group[c_group["nodegree"]==1]

control_group_x_00 = control_group_x_0[control_group_x_0["black"]==0]
control_group_x_10 = control_group_x_1[control_group_x_1["black"]==0]
control_group_x_01 = control_group_x_0[control_group_x_0["black"]==1]
control_group_x_11 = control_group_x_1[control_group_x_1["black"]==1]


min_number_x_00 = min(len(treatment_group_x_00), len(control_group_x_00))
balanced_treatment_x_00 = treatment_group_x_00[0:min_number_x_00]
balanced_control_x_00 = control_group_x_00[0:min_number_x_00]
print("After balancing: number of people with nodegree=0 and black=0 in treatment: ", len(balanced_treatment_x_00)," and in control: ", len(balanced_control_x_00))

min_number_x_01 = min(len(treatment_group_x_01), len(control_group_x_01))
balanced_treatment_x_01 = treatment_group_x_01[0:min_number_x_01]
balanced_control_x_01 = control_group_x_01[0:min_number_x_01]
print("After balancing: number of people with nodegree=0 and black=1 in treatment: ", len(balanced_treatment_x_01)," and in control: ", len(balanced_control_x_01))

min_number_x_10 = min(len(treatment_group_x_10), len(control_group_x_10))
balanced_treatment_x_10 = treatment_group_x_10[0:min_number_x_10]
balanced_control_x_10 = control_group_x_10[0:min_number_x_10]
print("After balancing: number of people with nodegree=1 and black=0 in treatment: ", len(balanced_treatment_x_10)," and in control: ", len(balanced_control_x_10))

min_number_x_11 = min(len(treatment_group_x_11), len(control_group_x_11))
balanced_treatment_x_11 = treatment_group_x_11[0:min_number_x_11]
balanced_control_x_11 = control_group_x_11[0:min_number_x_11]
print("After balancing: number of people with nodegree=1 and black=1 in treatment: ", len(balanced_treatment_x_11)," and in control: ", len(balanced_control_x_11))


After balancing: number of people with nodegree=0 and black=0 in treatment:  18  and in control:  18
After balancing: number of people with nodegree=0 and black=1 in treatment:  61  and in control:  61
After balancing: number of people with nodegree=1 and black=0 in treatment:  41  and in control:  41
After balancing: number of people with nodegree=1 and black=1 in treatment:  176  and in control:  176


In [26]:
balanced_treatment2 = pd.concat([balanced_treatment_x_00, balanced_treatment_x_01, balanced_treatment_x_10, balanced_treatment_x_11])
balanced_control2 = pd.concat([balanced_control_x_01, balanced_control_x_00, balanced_control_x_11, balanced_control_x_10])

ATE_matching2 = np.mean(balanced_treatment2["re78"]) - np.mean(balanced_control2["re78"])
print("ATE after balancing:", ATE_matching2)

ATE after balancing: 624.4923380405407


In [40]:
treatment_group_x_000 = treatment_group_x_00[treatment_group_x_00["hispanic"]==0]
treatment_group_x_001 = treatment_group_x_00[treatment_group_x_00["hispanic"]==1]
treatment_group_x_100 = treatment_group_x_10[treatment_group_x_10["hispanic"]==0]
treatment_group_x_101 = treatment_group_x_10[treatment_group_x_10["hispanic"]==1]
treatment_group_x_011 = treatment_group_x_01[treatment_group_x_01["hispanic"]==1]
treatment_group_x_010 = treatment_group_x_01[treatment_group_x_01["hispanic"]==0]
treatment_group_x_111 = treatment_group_x_11[treatment_group_x_11["hispanic"]==1]
treatment_group_x_110 = treatment_group_x_11[treatment_group_x_11["hispanic"]==0]


control_group_x_000 = control_group_x_00[control_group_x_00["hispanic"]==0]
control_group_x_001 = control_group_x_00[control_group_x_00["hispanic"]==1]
control_group_x_100 = control_group_x_10[control_group_x_10["hispanic"]==0]
control_group_x_101 = control_group_x_10[control_group_x_10["hispanic"]==1]
control_group_x_011 = control_group_x_01[control_group_x_01["hispanic"]==1]
control_group_x_010 = control_group_x_01[control_group_x_01["hispanic"]==0]
control_group_x_111 = control_group_x_11[control_group_x_11["hispanic"]==1]
control_group_x_110 = control_group_x_11[control_group_x_11["hispanic"]==0]

min_number_x_000 = min(len(treatment_group_x_000), len(control_group_x_000))
balanced_treatment_x_000 = treatment_group_x_000[0:min_number_x_000]
balanced_control_x_000 = control_group_x_000[0:min_number_x_000]
print("After balancing: number of people with nodegree=0 and black=0 and hispanic=0 in treatment: ", len(balanced_treatment_x_000)," and in control: ", len(balanced_control_x_000))

min_number_x_001 = min(len(treatment_group_x_001), len(control_group_x_001))
balanced_treatment_x_001 = treatment_group_x_001[0:min_number_x_001]
balanced_control_x_001 = control_group_x_001[0:min_number_x_001]
print("After balancing: number of people with nodegree=0 and black=0 and hispanic=1 in treatment: ", len(balanced_treatment_x_001)," and in control: ", len(balanced_control_x_001))



min_number_x_011 = min(len(treatment_group_x_011), len(control_group_x_011))
balanced_treatment_x_011 = treatment_group_x_011[0:min_number_x_011]
balanced_control_x_011 = control_group_x_011[0:min_number_x_011]
print("After balancing: number of people with nodegree=0 and black=1 hispanic=1 in treatment: ", len(balanced_treatment_x_011)," and in control: ", len(balanced_control_x_011))

min_number_x_010 = min(len(treatment_group_x_010), len(control_group_x_010))
balanced_treatment_x_010 = treatment_group_x_010[0:min_number_x_010]
balanced_control_x_010 = control_group_x_010[0:min_number_x_010]
print("After balancing: number of people with nodegree=0 and black=1 hispanic=0 in treatment: ", len(balanced_treatment_x_010)," and in control: ", len(balanced_control_x_010))



min_number_x_100 = min(len(treatment_group_x_100), len(control_group_x_100))
balanced_treatment_x_100 = treatment_group_x_100[0:min_number_x_100]
balanced_control_x_100 = control_group_x_100[0:min_number_x_100]
print("After balancing: number of people with nodegree=1 and black=0 hispanic=0 in treatment: ", len(balanced_treatment_x_100)," and in control: ", len(balanced_control_x_100))

min_number_x_101 = min(len(treatment_group_x_101), len(control_group_x_101))
balanced_treatment_x_101 = treatment_group_x_101[0:min_number_x_101]
balanced_control_x_101 = control_group_x_101[0:min_number_x_101]
print("After balancing: number of people with nodegree=1 and black=0 hispanic=1 in treatment: ", len(balanced_treatment_x_101)," and in control: ", len(balanced_control_x_101))

min_number_x_111 = min(len(treatment_group_x_111), len(control_group_x_111))
balanced_treatment_x_111 = treatment_group_x_111[0:min_number_x_111]
balanced_control_x_111 = control_group_x_111[0:min_number_x_111]
print("After balancing: number of people with nodegree=1 and black=1 hispanic=1 in treatment: ", len(balanced_treatment_x_111)," and in control: ", len(balanced_control_x_111))

min_number_x_110 = min(len(treatment_group_x_110), len(control_group_x_110))
balanced_treatment_x_110 = treatment_group_x_110[0:min_number_x_110]
balanced_control_x_110 = control_group_x_110[0:min_number_x_110]
print("After balancing: number of people with nodegree=1 and black=1 hispanic=0 in treatment: ", len(balanced_treatment_x_110)," and in control: ", len(balanced_control_x_110))


After balancing: number of people with nodegree=0 and black=0 and hispanic=0 in treatment:  13  and in control:  13
After balancing: number of people with nodegree=0 and black=0 and hispanic=1 in treatment:  4  and in control:  4
After balancing: number of people with nodegree=0 and black=1 hispanic=1 in treatment:  0  and in control:  0
After balancing: number of people with nodegree=0 and black=1 hispanic=0 in treatment:  61  and in control:  61
After balancing: number of people with nodegree=1 and black=0 hispanic=0 in treatment:  17  and in control:  17
After balancing: number of people with nodegree=1 and black=0 hispanic=1 in treatment:  24  and in control:  24
After balancing: number of people with nodegree=1 and black=1 hispanic=1 in treatment:  0  and in control:  0
After balancing: number of people with nodegree=1 and black=1 hispanic=0 in treatment:  176  and in control:  176


In [41]:
balanced_treatment2 = pd.concat([balanced_treatment_x_000, balanced_treatment_x_001, balanced_treatment_x_010, balanced_treatment_x_011,
                                 balanced_treatment_x_100,balanced_treatment_x_101,balanced_treatment_x_110,balanced_treatment_x_111])
balanced_control2 = pd.concat([balanced_control_x_000, balanced_control_x_001, balanced_control_x_010, balanced_control_x_011,
                               balanced_control_x_100, balanced_control_x_101, balanced_control_x_110, balanced_control_x_111,
                               ])

ATE_matching2 = np.mean(balanced_treatment2["re78"]) - np.mean(balanced_control2["re78"])
print("ATE after balancing:", ATE_matching2)

ATE after balancing: 685.5759391864403


Exact matching discarded a lot of data, so the estimate is quite unstable. We now look quickly at propensity scores, which are again not very useful in this case, and then we look at inverse probability weighting which does not discard any data.

## Propensity scores

We now compute $P(T=1|X)$ for each value of $X$:

In [45]:
x000_group = pd.concat([treatment_group_x_000,control_group_x_000])
x001_group = pd.concat([treatment_group_x_001,control_group_x_001])
x010_group = pd.concat([treatment_group_x_010,control_group_x_010])
x011_group = pd.concat([treatment_group_x_011,control_group_x_011])

x100_group = pd.concat([treatment_group_x_100,control_group_x_100])
x101_group = pd.concat([treatment_group_x_101,control_group_x_101])
x110_group = pd.concat([treatment_group_x_110,control_group_x_110])
x111_group = pd.concat([treatment_group_x_111,control_group_x_111])


propensity_score_x000 = len(treatment_group_x_000)/len(x000_group)
propensity_score_x001 = len(treatment_group_x_001)/len(x001_group)
propensity_score_x010 = len(treatment_group_x_010)/len(x010_group)
#propensity_score_x011 = len(treatment_group_x_011)/len(x011_group)


propensity_score_x100 = len(treatment_group_x_100)/len(x100_group)
propensity_score_x101 = len(treatment_group_x_101)/len(x101_group)
propensity_score_x110 = len(treatment_group_x_110)/len(x110_group)
#propensity_score_x111 = len(treatment_group_x_111)/len(x111_group)

We could do matching on the values of propensity scores (this is called propensity score matching), but in this case it doesn't really improve things since the probabilities are quite different for every combination of X. Otherwise it can be seen as a way of clustering values of X that have a similar effect on T.

## Inverse probability weighting

In [46]:
number_of_all_samples = len(df2)

weighted_sum_treatment_000 = sum(treatment_group_x_000["re78"])/  propensity_score_x000
weighted_sum_treatment_001 = sum(treatment_group_x_001["re78"])/  propensity_score_x001
weighted_sum_treatment_010 = sum(treatment_group_x_010["re78"])/ propensity_score_x010
weighted_sum_treatment_100 = sum(treatment_group_x_100["re78"])/ propensity_score_x100
weighted_sum_treatment_101 = sum(treatment_group_x_101["re78"])/ propensity_score_x101
weighted_sum_treatment_110 = sum(treatment_group_x_110["re78"])/ propensity_score_x110


mean_treatment2 = weighted_sum_treatment_000 + weighted_sum_treatment_001 + weighted_sum_treatment_010 + weighted_sum_treatment_100 + weighted_sum_treatment_101 + weighted_sum_treatment_110
mean_treatment2 = mean_treatment2/number_of_all_samples

weighted_sum_control_000 = sum(control_group_x_000["re78"])/(1 - propensity_score_x000)
weighted_sum_control_010 = sum(control_group_x_010["re78"])/ (1 - propensity_score_x010)
weighted_sum_control_001 = sum(control_group_x_001["re78"])/ (1 - propensity_score_x001)
weighted_sum_control_100 = sum(control_group_x_100["re78"])/ (1 - propensity_score_x100)
weighted_sum_control_101 = sum(control_group_x_101["re78"])/ (1 - propensity_score_x101)
weighted_sum_control_110 = sum(control_group_x_110["re78"])/ (1 - propensity_score_x110)

mean_control2 = weighted_sum_control_000 + weighted_sum_control_001 + weighted_sum_control_010 + weighted_sum_control_100 + weighted_sum_control_101 + weighted_sum_control_110
mean_control2 = mean_control2/number_of_all_samples

ATE_IPW2 = mean_treatment2-mean_control2
print("ATE after IPW:", ATE_IPW2)

ATE after IPW: 783.4030691284797
